In [81]:
import shared.calendar_utilities as cu
import get_price.get_futures_price as gfp
import contract_utilities.contract_meta_info as cmi
import fundamental_data.cot_data as cot
import shared.statistics as stats
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

ticker_head = 'C'
date_to=20170303

ticker_class = cmi.ticker_class[ticker_head]

date5_years_ago = cu.doubledate_shift(date_to, 5*365)
datetime_to = cu.convert_doubledate_2datetime(date_to)

panel_data = gfp.get_futures_price_preloaded(ticker_head=ticker_head,settle_date_from=date5_years_ago,settle_date_to=date_to)
panel_data = panel_data[panel_data['tr_dte']>=40]
panel_data.sort(['settle_date','tr_dte'],ascending=[True,True],inplace=True)
rolling_data = panel_data.drop_duplicates(subset=['settle_date'], take_last=False)

daily_noise = np.std(rolling_data['change_1'])
average_volume = rolling_data['volume'].mean()

rolling_data['change_40'] =  pd.rolling_sum(rolling_data['change_1'], 40, min_periods=30)
rolling_data['change_20'] =  pd.rolling_sum(rolling_data['change_1'], 20, min_periods=15)
rolling_data['change_10'] =  pd.rolling_sum(rolling_data['change_1'], 10, min_periods=7)



rolling_data['change_1Normalized'] = rolling_data['change_1']/daily_noise
rolling_data['change_5Normalized'] = rolling_data['change_5']/daily_noise
rolling_data['change_10Normalized'] = rolling_data['change_10']/daily_noise
rolling_data['change_20Normalized'] = rolling_data['change_20']/daily_noise
rolling_data['change_40Normalized'] = rolling_data['change_40']/daily_noise

rolling_data['change1Normalized'] = rolling_data['change1']/daily_noise
rolling_data['change1_InstantNormalized'] = rolling_data['change1_instant']/daily_noise
rolling_data['change5Normalized'] = rolling_data['change5']/daily_noise
rolling_data['change10Normalized'] = rolling_data['change10']/daily_noise
rolling_data['change20Normalized'] = rolling_data['change20']/daily_noise

rolling_data['volume_mean20'] = pd.rolling_mean(rolling_data['volume'], 20, min_periods=15)
rolling_data['volume_mean10'] = pd.rolling_mean(rolling_data['volume'], 10, min_periods=7)
rolling_data['volume_mean5'] = pd.rolling_mean(rolling_data['volume'], 5, min_periods=4)

rolling_data['volume_mean20Normalized'] = rolling_data['volume_mean20']/average_volume
rolling_data['volume_mean10Normalized'] = rolling_data['volume_mean10']/average_volume
rolling_data['volume_mean5Normalized'] = rolling_data['volume_mean5']/average_volume
rolling_data['volume_Normalized'] = rolling_data['volume']/average_volume

rolling_data['change_1_high_volume'] = (rolling_data['volume']>average_volume)*rolling_data['change_1Normalized']
rolling_data['change_5_high_volume'] = (rolling_data['volume_mean5']>average_volume)*rolling_data['change_5Normalized']
rolling_data['change_10_high_volume'] = (rolling_data['volume_mean10']>average_volume)*rolling_data['change_10Normalized']
rolling_data['change_20_high_volume'] = (rolling_data['volume_mean20']>average_volume)*rolling_data['change_20Normalized']

rolling_data['change_1_low_volume'] = (rolling_data['volume']<=average_volume)*rolling_data['change_1Normalized']
rolling_data['change_5_low_volume'] = (rolling_data['volume_mean5']<=average_volume)*rolling_data['change_5Normalized']
rolling_data['change_10_low_volume'] = (rolling_data['volume_mean10']<=average_volume)*rolling_data['change_10Normalized']
rolling_data['change_20_low_volume'] = (rolling_data['volume_mean20']<=average_volume)*rolling_data['change_20Normalized']

cot_output = cot.get_cot_data(ticker_head=ticker_head,date_from=date5_years_ago,date_to=date_to)

cot_net = pd.DataFrame()

if ticker_class in ['FX','STIR','Index','Treasury']:
    cot_net['comm_net'] = cot_output['Dealer Longs']-cot_output['Dealer Shorts']
    cot_net['spec_net'] = cot_output['Asset Manager Longs']-cot_output['Asset Manager Shorts']+cot_output['Leveraged Funds Longs']-cot_output['Leveraged Funds Shorts']
else:
    cot_net['comm_net'] = cot_output['Producer/Merchant/Processor/User Longs']-cot_output['Producer/Merchant/Processor/User Shorts']
    cot_net['spec_net'] = cot_output['Money Manager Longs']-cot_output['Money Manager Shorts']

cot_net['comm_net_change_1'] = cot_net['comm_net']-cot_net['comm_net'].shift(1)
cot_net['comm_net_change_2'] = cot_net['comm_net']-cot_net['comm_net'].shift(2)
cot_net['comm_net_change_4'] = cot_net['comm_net']-cot_net['comm_net'].shift(4)

cot_net['spec_net_change_1'] = cot_net['spec_net']-cot_net['spec_net'].shift(1)
cot_net['spec_net_change_2'] = cot_net['spec_net']-cot_net['spec_net'].shift(2)
cot_net['spec_net_change_4'] = cot_net['spec_net']-cot_net['spec_net'].shift(4)

comm_net_change_1_avg = np.std(cot_net['comm_net_change_1'])
comm_net_change_2_avg = np.std(cot_net['comm_net_change_2'])
comm_net_change_4_avg = np.std(cot_net['comm_net_change_4'])
spec_net_change_1_avg = np.std(cot_net['spec_net_change_1'])
spec_net_change_2_avg = np.std(cot_net['spec_net_change_2'])
spec_net_change_4_avg = np.std(cot_net['spec_net_change_4'])

cot_net['comm_net_change_1_normalized']  = cot_net['comm_net_change_1']/comm_net_change_1_avg
cot_net['comm_net_change_2_normalized'] = cot_net['comm_net_change_2']/comm_net_change_2_avg
cot_net['comm_net_change_4_normalized'] = cot_net['comm_net_change_4']/comm_net_change_4_avg

cot_net['spec_net_change_1_normalized']  = cot_net['spec_net_change_1']/spec_net_change_1_avg
cot_net['spec_net_change_2_normalized'] = cot_net['spec_net_change_2']/spec_net_change_2_avg
cot_net['spec_net_change_4_normalized'] = cot_net['spec_net_change_4']/spec_net_change_4_avg

cot_net['comm_z']= (cot_net['comm_net']-np.mean(cot_net['comm_net']))/np.std(cot_net['comm_net'])
cot_net['spec_z']= (cot_net['spec_net']-np.mean(cot_net['spec_net']))/np.std(cot_net['spec_net'])

cot_net['settle_date'] = cot_net.index
cot_net['settle_date'] = [x+dt.timedelta(days=3) for x in cot_net['settle_date']]  

combined_data = pd.merge(rolling_data,cot_net,how='outer',on='settle_date')

combined_data = pd.merge(rolling_data,cot_net,how='left',on='settle_date')

combined_data['comm_net_change_1_normalized']  = combined_data['comm_net_change_1_normalized'].fillna(method='pad')
combined_data['comm_net_change_2_normalized']  = combined_data['comm_net_change_2_normalized'].fillna(method='pad')
combined_data['comm_net_change_4_normalized']  = combined_data['comm_net_change_4_normalized'].fillna(method='pad')

combined_data['spec_net_change_1_normalized']  = combined_data['spec_net_change_1_normalized'].fillna(method='pad')
combined_data['spec_net_change_2_normalized']  = combined_data['spec_net_change_2_normalized'].fillna(method='pad')
combined_data['spec_net_change_4_normalized']  = combined_data['spec_net_change_4_normalized'].fillna(method='pad')

combined_data['comm_z']  = combined_data['comm_z'].fillna(method='pad')
combined_data['spec_z']  = combined_data['spec_z'].fillna(method='pad')

test_data = combined_data[combined_data['settle_date'] == datetime_to]
training_data = combined_data[combined_data['settle_date'] < datetime_to+dt.timedelta(days=-30)]


indicator_list = ['change_1_high_volume','change_5_high_volume','change_10_high_volume','change_20_high_volume',
                  'change_1_low_volume','change_5_low_volume','change_10_low_volume','change_20_low_volume',
                 'comm_net_change_1_normalized','comm_net_change_2_normalized','comm_net_change_4_normalized',
                 'spec_net_change_1_normalized','spec_net_change_2_normalized','spec_net_change_4_normalized',
                 'comm_z','spec_z']

sharp1_list = []
sharp1_instant_list = []
sharp5_list = []
sharp10_list = []
sharp20_list = []

for i in range(len(indicator_list)):
    selected_data = training_data[training_data[indicator_list[i]].notnull()]
    indicator_levels = stats.get_number_from_quantile(y=selected_data[indicator_list[i]].values,quantile_list=[10,90])
    low_data = selected_data[selected_data[indicator_list[i]]<indicator_levels[0]]
    high_data = selected_data[selected_data[indicator_list[i]]>indicator_levels[1]]
    high_data['pnl1'] = high_data['change1Normalized']
    low_data['pnl1'] = -low_data['change1Normalized']

    high_data['pnl1_instant'] = high_data['change1_InstantNormalized']
    low_data['pnl1_instant'] = -low_data['change1_InstantNormalized']

    high_data['pnl5'] = high_data['change5Normalized']
    low_data['pnl5'] = -low_data['change5Normalized']

    high_data['pnl10'] = high_data['change10Normalized']
    low_data['pnl10'] = -low_data['change10Normalized']

    high_data['pnl20'] = high_data['change20Normalized']
    low_data['pnl20'] = -low_data['change20Normalized']
    merged_data = pd.concat([high_data,low_data])
    sharp1_list.append(16*merged_data['pnl1'].mean()/merged_data['pnl1'].std())
    sharp1_instant_list.append(16*merged_data['pnl1_instant'].mean()/merged_data['pnl1_instant'].std())
    sharp5_list.append(7.2*merged_data['pnl5'].mean()/merged_data['pnl5'].std())
    sharp10_list.append(5.1*merged_data['pnl10'].mean()/merged_data['pnl10'].std())
    sharp20_list.append(3.5*merged_data['pnl20'].mean()/merged_data['pnl20'].std())

pd.DataFrame.from_items([('indicator', indicator_list), ('sharp1', sharp1_list),
                         ('sharp1_instant', sharp1_instant_list),
                         ('sharp5', sharp5_list),
                         ('sharp10', sharp10_list),
                         ('sharp20', sharp20_list)])  

vote1 = 0

for i in range(len(indicator_list)):
        indicator_value = test_data[indicator_list[i]].iloc[0]
        selected_sharp_row = sharp_frame[sharp_frame['indicator']==indicator_list[i]]

        if (selected_sharp_row['sharp1'].iloc[0]>0.75)&(indicator_value>selected_sharp_row['higher_level'].iloc[0]):
            vote1 += 1
        elif (selected_sharp_row['sharp1'].iloc[0]>0.75)&(indicator_value<selected_sharp_row['lower_level'].iloc[0]):
            vote1 -= 1
        elif (selected_sharp_row['sharp1'].iloc[0]<-0.75)&(indicator_value>selected_sharp_row['higher_level'].iloc[0]):
            vote1 -= 1
        elif (selected_sharp_row['sharp1'].iloc[0]<-0.75)&(indicator_value<selected_sharp_row['lower_level'].iloc[0]):
            vote1 += 1

vote1_instant = 0

for i in range(len(indicator_list)):
        indicator_value = test_data[indicator_list[i]].iloc[0]
        selected_sharp_row = sharp_frame[sharp_frame['indicator']==indicator_list[i]]

        if (selected_sharp_row['sharp1'].iloc[0]>0.75)&(indicator_value>selected_sharp_row['higher_level'].iloc[0]):
            vote1 += 1
        elif (selected_sharp_row['sharp1'].iloc[0]>0.75)&(indicator_value<selected_sharp_row['lower_level'].iloc[0]):
            vote1 -= 1
        elif (selected_sharp_row['sharp1'].iloc[0]<-0.75)&(indicator_value>selected_sharp_row['higher_level'].iloc[0]):
            vote1 -= 1
        elif (selected_sharp_row['sharp1'].iloc[0]<-0.75)&(indicator_value<selected_sharp_row['lower_level'].iloc[0]):
            vote1 += 1            

,indicator,sharp1,sharp1_instant,sharp5,sharp10,sharp20
0,change_1_high_volume,-2.272068,-0.354479,-1.096913,-0.206210,-0.318771
1,change_5_high_volume,-0.923404,-0.738286,0.242040,0.456185,-0.224115
2,change_10_high_volume,-0.178115,-0.486540,0.438771,0.647699,-0.615364
3,change_20_high_volume,-0.867102,-0.527076,-0.745598,-0.973863,-0.858538
4,change_1_low_volume,1.002939,1.256766,0.166871,-0.004207,0.194610
5,change_5_low_volume,0.248088,0.171210,0.758249,0.426577,0.326980
6,change_10_low_volume,-0.288082,0.878143,0.713787,0.680334,0.390458
7,change_20_low_volume,0.968820,1.256268,0.758662,0.680382,0.543490
8,comm_net_change_1_normalized,0.044250,0.235797,-0.333442,0.237796,0.182879
9,comm_net_change_2_normalized,-0.207515,0.197873,-0.465053,-0.269089,-0.434211


In [82]:
test_data



,settle_date,ticker,ticker_head,ticker_month,ticker_year,cal_dte,tr_dte,open_price,high_price,low_price,...,spec_net_change_2,spec_net_change_4,comm_net_change_1_normalized,comm_net_change_2_normalized,comm_net_change_4_normalized,spec_net_change_1_normalized,spec_net_change_2_normalized,spec_net_change_4_normalized,comm_z,spec_z
1259,2017-03-03,CK2017,C,5,2017,70,49,378,381.75,377.5,...,-3225,78674,-0.106905,-0.325297,-1.20121,-0.282734,-0.054453,0.840282,-1.343993,0.210455


In [20]:
import fundamental_data.cot_data as cot
import get_price.get_futures_price as gfp
import contract_utilities.contract_meta_info as cmi
import datetime as dt

import numpy as np

import pandas as pd
date_to=20170303
ticker_head='CL'
cot_output = cot.get_cot_data(ticker_head=ticker_head,date_to=date_to)

ticker_class = cmi.ticker_class[ticker_head]

cot_net = pd.DataFrame()

if ticker_class in ['FX','STIR','Index','Treasury']:
    cot_net['comm_net'] = cot_output['Dealer Longs']-cot_output['Dealer Shorts']
    cot_net['spec_net'] = cot_output['Asset Manager Longs']-cot_output['Asset Manager Shorts']+cot_output['Leveraged Funds Longs']-cot_output['Leveraged Funds Shorts']
else:
    cot_net['comm_net'] = cot_output['Producer/Merchant/Processor/User Longs']-cot_output['Producer/Merchant/Processor/User Shorts']
    cot_net['spec_net'] = cot_output['Money Manager Longs']-cot_output['Money Manager Shorts']


data_out = gfp.get_futures_price_preloaded(ticker_head=ticker_head)
data_out = data_out[data_out['tr_dte']>30]
data_out.drop_duplicates(subset=['settle_date'], take_last=False, inplace=True)
data_out['change20Percent'] = 100*data_out['change20']/data_out['close_price']
data_out['change5Percent'] = 100*data_out['change5']/data_out['close_price']
data_out = data_out[['settle_date','change5Percent','change20Percent']]

cot_net['comm_net_change_1'] = cot_net['comm_net']-cot_net['comm_net'].shift(1)
cot_net['comm_net_change_2'] = cot_net['comm_net']-cot_net['comm_net'].shift(2)
cot_net['comm_net_change_4'] = cot_net['comm_net']-cot_net['comm_net'].shift(4)

cot_net['spec_net_change_1'] = cot_net['spec_net']-cot_net['spec_net'].shift(1)
cot_net['spec_net_change_2'] = cot_net['spec_net']-cot_net['spec_net'].shift(2)
cot_net['spec_net_change_4'] = cot_net['spec_net']-cot_net['spec_net'].shift(4)

comm_net_change_1_avg = np.std(cot_net['comm_net_change_1'])
comm_net_change_2_avg = np.std(cot_net['comm_net_change_2'])
comm_net_change_4_avg = np.std(cot_net['comm_net_change_4'])
spec_net_change_1_avg = np.std(cot_net['spec_net_change_1'])
spec_net_change_2_avg = np.std(cot_net['spec_net_change_2'])
spec_net_change_4_avg = np.std(cot_net['spec_net_change_4'])

cot_net['comm_net_change_1_normalized']  = cot_net['comm_net_change_1']/comm_net_change_1_avg
cot_net['comm_net_change_2_normalized'] = cot_net['comm_net_change_2']/comm_net_change_2_avg
cot_net['comm_net_change_4_normalized'] = cot_net['comm_net_change_4']/comm_net_change_4_avg

cot_net['spec_net_change_1_normalized']  = cot_net['spec_net_change_1']/spec_net_change_1_avg
cot_net['spec_net_change_2_normalized'] = cot_net['spec_net_change_2']/spec_net_change_2_avg
cot_net['spec_net_change_3_normalized'] = cot_net['spec_net_change_4']/spec_net_change_4_avg

cot_net['comm_z']= (cot_net['comm_net']-np.mean(cot_net['comm_net']))/np.std(cot_net['comm_net'])
cot_net['spec_z']= (cot_net['spec_net']-np.mean(cot_net['spec_net']))/np.std(cot_net['spec_net'])

cot_net['settle_date'] = cot_net.index
cot_net['settle_date'] = [x+dt.timedelta(days=3) for x in cot_net['settle_date']]

merged_data = pd.merge(data_out,cot_net,how='inner',on='settle_date')

In [21]:
import shared.statistics as stats
import pandas as pd
indicator_list = ['comm_net_change_1_normalized','comm_net_change_2_normalized','comm_net_change_4_normalized',
                 'spec_net_change_1_normalized','spec_net_change_2_normalized','spec_net_change_3_normalized',
                 'comm_z','spec_z']

monthly_sharp_list =[]
weekly_sharp_list = []

for i in range(len(indicator_list)):
    indicator_levels = stats.get_number_from_quantile(y=merged_data[indicator_list[i]].values,quantile_list=[10,90])
    low_data = merged_data[merged_data[indicator_list[i]]<indicator_levels[0]]
    high_data = merged_data[merged_data[indicator_list[i]]>indicator_levels[1]]
    monthly_sharp_list.append(3.5*high_data['change20Percent'].mean()/high_data['change20Percent'].std()-
                      3.5*low_data['change20Percent'].mean()/low_data['change20Percent'].std())
    weekly_sharp_list.append(7*high_data['change5Percent'].mean()/high_data['change5Percent'].std()-
                     7*low_data['change5Percent'].mean()/low_data['change5Percent'].std())


pd.DataFrame.from_items([('indicator', indicator_list), ('monthly_sharp', monthly_sharp_list),
                        ('weekly_sharp', weekly_sharp_list)])    

# LN: spec_change positively spec_z negatively correlated
# LC: spec_change positively spec_z positively correlated
# FC: spec_change positively correlated
# ES: spec_change negatively correlated
# NQ: spec_change nagatively correlated
# EC: comm_change positively correlated but comm_z is negative
# JY: week relationships
# AD: comm_change positively correlated
# CD: comm_change positively correlated
# BP: weak relationships
# GC: spec_change nagatively correlated
# SI: weak relationships
# TY: spec_change positively correlated
# US: weak relationships
# FV: comm_change positively correlated
# TU: weak relationships
# C: spec_change positively correlated
# S: spec_change positively correlated
# SM: spec_change positively correlated
# BO: comm_change positively correlated
# W: spec_change positively correlated
# KW_ spec_change positively correlated





,indicator,monthly_sharp,weekly_sharp
0,comm_net_change_1_normalized,0.491848,-0.268108
1,comm_net_change_2_normalized,0.560898,0.160695
2,comm_net_change_4_normalized,1.207450,2.231186
3,spec_net_change_1_normalized,-0.182198,1.116041
4,spec_net_change_2_normalized,-0.853351,0.791972
5,spec_net_change_3_normalized,0.079188,-0.633236
6,comm_z,-0.015113,0.155466
7,spec_z,0.213558,-0.306237


In [23]:
import get_price.get_futures_price as gfp
data_out = gfp.get_futures_price_preloaded(ticker_head='C')
data_out.columns

Index(['settle_date', 'ticker', 'ticker_head', 'ticker_month', 'ticker_year',
       'cal_dte', 'tr_dte', 'open_price', 'high_price', 'low_price',
       'close_price', 'volume', 'cont_indx', 'change1', 'change1_instant',
       'change2', 'change5', 'change10', 'change20', 'change_5', 'change_1'],
      dtype='object')

In [81]:
long_data = merged_data[merged_data['comm_net_delta_4']>0]
short_data = merged_data[merged_data['comm_net_delta_4']<0]

print(7*long_data['change5Percent'].mean()/long_data['change5Percent'].std())
print(7*short_data['change5Percent'].mean()/short_data['change5Percent'].std())

0.31993542926693
0.0825970614781219


In [82]:


import matplotlib.pyplot as plt

plt.plot(long_data['change5Percent'].cumsum())
plt.grid()         
         
plt.show()

In [56]:
import backtesting.utilities as bu
dates_output = bu.get_backtesting_dates(date_to=20170303,years_back=10,day_of_week=4)
double_dates = sorted(dates_output['double_dates'],reverse=False)
selected_dates = double_dates
selected_dates

[20070309,
 20070316,
 20070323,
 20070330,
 20070413,
 20070420,
 20070427,
 20070504,
 20070511,
 20070518,
 20070525,
 20070601,
 20070608,
 20070615,
 20070622,
 20070629,
 20070706,
 20070713,
 20070720,
 20070727,
 20070803,
 20070810,
 20070817,
 20070824,
 20070831,
 20070907,
 20070914,
 20070921,
 20070928,
 20071005,
 20071012,
 20071019,
 20071026,
 20071102,
 20071109,
 20071116,
 20071123,
 20071130,
 20071207,
 20071214,
 20071221,
 20071228,
 20080104,
 20080111,
 20080118,
 20080125,
 20080201,
 20080208,
 20080215,
 20080222,
 20080229,
 20080307,
 20080314,
 20080328,
 20080404,
 20080411,
 20080418,
 20080425,
 20080502,
 20080509,
 20080516,
 20080523,
 20080530,
 20080606,
 20080613,
 20080620,
 20080627,
 20080711,
 20080718,
 20080725,
 20080801,
 20080808,
 20080815,
 20080822,
 20080829,
 20080905,
 20080912,
 20080919,
 20080926,
 20081003,
 20081010,
 20081017,
 20081024,
 20081031,
 20081107,
 20081114,
 20081121,
 20081128,
 20081205,
 20081212,
 20081219,

In [57]:
import opportunity_constructs.cot_strategy as cots
import pandas as pd

backtest_results = []

for i in range(len(selected_dates)):
    print(selected_dates[i])
    cot_output = cots.get_cot_sheet_4date(date_to=selected_dates[i])
    if cot_output['success']:
        cot_sheet = cot_output['cot_sheet']
        cot_sheet['settle_date'] = selected_dates[i]
        backtest_results.append(cot_sheet)
        

backtest_frame = pd.concat(backtest_results)

20070309
20070316
20070323
20070330
20070413
20070420
20070427
20070504
20070511
20070518
20070525
20070601
20070608
20070615
20070622
20070629
20070706
20070713
20070720
20070727
20070803
20070810
20070817
20070824
20070831
20070907
20070914
20070921
20070928
20071005
20071012
20071019
20071026
20071102
20071109
20071116
20071123
20071130
20071207
20071214
20071221
20071228
20080104
20080111
20080118
20080125
20080201
20080208
20080215
20080222
20080229
20080307
20080314
20080328
20080404
20080411
20080418
20080425
20080502
20080509
20080516
20080523
20080530
20080606
20080613
20080620
20080627
20080711
20080718
20080725
20080801
20080808
20080815
20080822
20080829
20080905
20080912
20080919
20080926
20081003
20081010
20081017
20081024
20081031
20081107
20081114
20081121
20081128
20081205
20081212
20081219
20081226
20090102
20090109
20090116
20090123
20090130
20090206
20090213
20090220
20090227
20090306
20090313
20090320
20090327
20090403
20090417
20090424
20090501
20090508
20090515
2

In [58]:
backtest_frame['normalized_pnl_5_'] = -backtest_frame['normalized_pnl_5']
backtest_frame['normalized_pnl_10_'] = -backtest_frame['normalized_pnl_10']
backtest_frame['normalized_pnl_20_'] = -backtest_frame['normalized_pnl_20']

In [80]:
backtest_frame.columns

Index(['ticker', 'ticker_head', 'ticker_class', 'change_5_normalized',
       'change_10_normalized', 'change_20_normalized', 'change_40_normalized',
       'change_60_normalized', 'comm_net_change_1_normalized',
       'comm_net_change_2_normalized', 'comm_net_change_4_normalized',
       'spec_net_change_1_normalized', 'spec_net_change_2_normalized',
       'spec_net_change_4_normalized', 'volume_5_normalized',
       'volume_10_normalized', 'volume_20_normalized', 'comm_z', 'spec_z',
       'normalized_pnl_5', 'normalized_pnl_10', 'normalized_pnl_20',
       'settle_date', 'normalized_pnl_5_', 'normalized_pnl_10_',
       'normalized_pnl_20_'],
      dtype='object')

In [84]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display
backtest_frame['tickerHead'] = backtest_frame['ticker_head']
rank_indicators_output = bs.rank_indicators(trade_data=backtest_frame,
                                            indicator_list = ['change_5_normalized',
                                                              'change_10_normalized', 'change_20_normalized', 'change_40_normalized',
                                                              'change_60_normalized', 'comm_net_change_1_normalized',
                                                              'comm_net_change_2_normalized', 'comm_net_change_4_normalized',
                                                              'spec_net_change_1_normalized', 'spec_net_change_2_normalized',
                                                              'spec_net_change_4_normalized', 'volume_5_normalized',
                                                              'volume_10_normalized', 'volume_20_normalized', 'comm_z', 'spec_z'],
                                            long_pnl_field='normalized_pnl_20',
                                            short_pnl_field='normalized_pnl_20_',
                                            strategy_class='cot',granular_ranking_type='ticker_head')
indicator_ranking_total = rank_indicators_output['indicator_ranking_total']
indicator_ranking_granular_total = rank_indicators_output['indicator_ranking_granular_total']
display(indicator_ranking_total)
display(indicator_ranking_granular_total)

,indicator,ranking
185,"[volume_5_normalized, change_60_normalized]",503
86,"[change_60_normalized, volume_5_normalized]",500
87,"[change_60_normalized, volume_10_normalized]",500
200,"[volume_10_normalized, change_60_normalized]",491
88,"[change_60_normalized, volume_20_normalized]",484
215,"[volume_20_normalized, change_60_normalized]",482
72,"[change_40_normalized, volume_10_normalized]",481
184,"[volume_5_normalized, change_40_normalized]",481
71,"[change_40_normalized, volume_5_normalized]",475
214,"[volume_20_normalized, change_40_normalized]",469


,indicator,ranking
185,"[volume_5_normalized, change_60_normalized]",10241
86,"[change_60_normalized, volume_5_normalized]",10184
184,"[volume_5_normalized, change_40_normalized]",9997
215,"[volume_20_normalized, change_60_normalized]",9976
73,"[change_40_normalized, volume_20_normalized]",9946
214,"[volume_20_normalized, change_40_normalized]",9942
200,"[volume_10_normalized, change_60_normalized]",9938
199,"[volume_10_normalized, change_40_normalized]",9871
192,"[volume_5_normalized, volume_10_normalized]",9830
88,"[change_60_normalized, volume_20_normalized]",9823


In [99]:
import backtesting.backtest_stats as bs
#selected_frame = backtest_frame[backtest_frame['volume_20_normalized']>2]

bs.get_indicator_rr_table(trade_data=backtest_frame,indicator_name='spec_z',
                          strategy_class='cot',
                          long_pnl_field='normalized_pnl_5',short_pnl_field='normalized_pnl_5_')

# some momentum for 5=>5, 5=>10, 5=>20
# some momentum for 10=>5, 10=>10, 10=>20
# some momentum for 20=>5, 20=>10, 20=>20
# some momentum for 40=>5, 40=>10, 40=>20 * these might be the strongest ones
# some momentum for 60=>5, 60=>10, 60=>20
# comm_net_change_x_normalized: buying with commercials does seem to have some merit, although sell side didn't work
# spec_net_change_x_normalized: buying with specs does seem to have some merit, although sell side didn't work
# spec_z: buying and selling with spec_z does seem to have some merit



,indicator_ulimit,mean_pnl,reward_risk
0,NaN,463.719847,0.048781
1,1.621625,656.638052,0.073328
2,1.057747,222.160851,0.023524
3,0.627131,300.733584,0.032967
4,0.221488,733.545252,0.082924
5,-0.180282,110.328814,0.011461
6,-0.604087,-213.740241,-0.021449
7,-1.020202,-533.996566,-0.058487
8,-1.498477,250.132733,0.027480


In [102]:
import backtesting.backtest_stats as bs
bs.get_indicator_rr_double_table(trade_data=backtest_frame,indicator_list=['change_40_normalized','spec_z'],
                                 strategy_class='cot',
                                 long_pnl_field='normalized_pnl_20',
                                            short_pnl_field='normalized_pnl_20_')

,indicator1_ulimit,indicator2_ulimit,mean_pnl,reward_risk
0,NaN,NaN,4429.023384,0.253837
1,NaN,0.960452,1229.284533,0.066232
2,NaN,-0.141262,1533.055436,0.088534
3,3.123034,NaN,1029.125938,0.060850
4,3.123034,0.562635,2289.403639,0.142067
5,3.123034,-0.560928,573.475652,0.035016
6,-2.425765,NaN,456.548758,0.022172
7,-2.425765,0.193429,1278.877150,0.068819
8,-2.425765,-0.968620,786.549869,0.044569


In [62]:
long_frame = backtest_frame[backtest_frame['change_40_normalized']<-8]
long_frame['normalized_pnl_20'].cumsum()



In [74]:
date_list = backtest_frame['settle_date'].unique()
import numpy as np
pnl_list = []

for i in range(len(date_list)):
    
    daily_data = backtest_frame[backtest_frame['settle_date']==date_list[i]]
    daily_data.sort('change_40_normalized',ascending=True,inplace=True)
    short_pnl = -np.sum(daily_data['normalized_pnl_5'].iloc[:3])
    long_pnl = np.sum(daily_data['normalized_pnl_5'].iloc[-3:])
    pnl_list.append(short_pnl+long_pnl)

In [75]:


import matplotlib.pyplot as plt

plt.plot(np.cumsum(pnl_list))
plt.grid()         
         
plt.show()

In [76]:
np.sqrt(52)*np.mean(pnl_list)/np.std(pnl_list)

0.33363369409048871

In [1]:
import fundamental_data.cot_data as cot
ticker_head = 'EC'
date_to = 20170102
cot_output = cot.get_cot_data(ticker_head=ticker_head,date_to=date_to)

cot_output

,Open Interest,Dealer Longs,Dealer Shorts,Dealer Spreads,Asset Manager Longs,Asset Manager Shorts,Asset Manager Spreads,Leveraged Funds Longs,Leveraged Funds Shorts,Leveraged Funds Spreads,Other Reportable Longs,Other Reportable Shorts,Other Reportable Spreads,Total Reportable Longs,Total Reportable Shorts,Non Reportable Longs,Non Reportable Shorts
Date,,,,,,,,,,,,,,,,,
2006-06-13,226641,34944,110033,28834,22352,17391,5138,47535,7570,9892,6523,2561,640,155856,182058,70785,44583
2006-06-20,156363,15888,74780,6617,21190,14822,4809,35719,12877,9504,7396,1531,984,102105,125923,54257,30440
2006-06-27,175206,16392,86793,8530,20435,14770,4606,49147,11270,11408,6296,3008,1171,117985,141555,57221,33651
2006-07-03,193017,15241,96729,11188,18375,14517,5740,62455,11419,12520,6018,2182,1483,133018,155777,59998,37239
2006-07-11,192637,13037,108341,9357,18281,13091,6115,67082,10531,9755,6211,3031,1070,130908,161291,61729,31346
2006-07-18,179294,10081,86189,11564,22961,11973,4578,48561,10085,10591,7289,2437,1159,116783,138576,62512,40719
2006-07-25,172685,12672,79376,12662,23021,11973,5033,44826,13073,10581,7014,3414,1315,117124,137426,55561,35259
2006-08-01,187609,15301,99895,9920,18412,11964,7169,58063,8152,12390,7306,2337,1823,130384,153650,57225,33959
2006-08-08,204673,17554,124168,9659,17314,12104,7347,72929,6681,9719,7767,1521,2313,144601,173511,60072,31161
